In [43]:
from faker import Faker
import pandas as pd

faker = Faker('pt_BR')

In [108]:
USER_AMOUNT = 3
user_names = [faker.name() for _ in range(USER_AMOUNT)]
emails = [faker.company_email() for _ in range(USER_AMOUNT)]

administrators = pd.DataFrame({
    "email": emails,
    "nome": user_names,
    "password": [faker.password() for _ in range(USER_AMOUNT)],
}, columns={"email": str, "nome": str, "password": str})
administrators.head()

,email,nome,password
0,joao-pedrorodrigues@vieira.br,Helena Costa,CCWmldaw+3
1,camposfernando@oliveira.net,Sofia Moraes,*yqxbZl649
2,vicentecampos@gomes.net,Sophia Cardoso,&0OB#FEobe


In [39]:
CONDOMINIUMS_AMOUNT = [faker.pyint(1, 10) for _ in range(USER_AMOUNT)]
TOTAL_APARTMENTS = sum(CONDOMINIUMS_AMOUNT)

condominiums_emails = []
for x in range(USER_AMOUNT):
    condominiums_emails += [emails[x]] * CONDOMINIUMS_AMOUNT[x]

condominiums = pd.DataFrame({
    "administradorEmail": condominiums_emails,
    "cnpj": [faker.cnpj() for _ in range(TOTAL_APARTMENTS)],
    "nome": [faker.company() for _ in range(TOTAL_APARTMENTS)],
    "address": [faker.address() for _ in range(TOTAL_APARTMENTS)],
    "valorMensalidade": [faker.pydecimal(left_digits=3, right_digits=0,
                                         positive=True, min_value=250
                        ) for _ in range(TOTAL_APARTMENTS)],
}, columns={"cnpj": str, "nome": str, "address": str,
            "valorMensalidade": float, "administradorEmail": str})

condominiums.head()

,cnpj,nome,address,valorMensalidade,administradorEmail
0,38.720.954/0001-41,das Neves,"Viaduto Alexia Barbosa, 74\nBarroca\n51950950 ...",250,pedro-lucas34@viana.br
1,08.617.254/0001-65,Lima Oliveira - ME,"Trecho de da Luz, 43\nPindorama\n88446490 Fari...",250,pedro-lucas34@viana.br
2,50.961.437/0001-60,Viana e Filhos,Praia Freitas\nTiradentes\n96923-801 Vieira da...,609,pedro-lucas34@viana.br
3,24.061.893/0001-43,Costa,"Viela João Lucas Fogaça, 416\nAeroporto\n71644...",987,pedro-lucas34@viana.br
4,96.487.132/0001-69,Teixeira da Mata e Filhos,"Rua Azevedo, 74\nVila Mangueiras\n99086-107 Co...",860,pedro-lucas34@viana.br


In [42]:
debtors = pd.DataFrame(columns={"rg": str, "cpf": str, "nome": str,
                                "telefone": str, "cnpjCondominio": str})

for index, row in condominiums.iterrows():
  DEBTORS_AMOUNT = faker.pyint(1, 20)
  debtors = pd.concat([debtors, pd.DataFrame({
    "rg": [faker.rg() for _ in range(DEBTORS_AMOUNT)],
    "cpf": [faker.cpf() for _ in range(DEBTORS_AMOUNT)],
    "nome": [faker.name() for _ in range(DEBTORS_AMOUNT)],
    "telefone": [faker.phone_number() for _ in range(DEBTORS_AMOUNT)],
    "cnpjCondominio": row["cnpj"]
  })], ignore_index=True)

debtors

,rg,cpf,nome,telefone,cnpjCondominio
0,423167054,768.320.195-21,Bruna da Rocha,+55 11 7882-3430,38.720.954/0001-41
1,142658078,276.905.413-99,Pedro Miguel Araújo,41 4628 6151,38.720.954/0001-41
2,204768354,530.782.619-21,Sabrina Santos,(041) 2219 7932,38.720.954/0001-41
3,631842755,197.642.350-34,Sra. Lara Lima,+55 (051) 3356 7235,38.720.954/0001-41
4,053267485,172.490.538-41,Letícia Duarte,+55 41 8011 6291,38.720.954/0001-41
...,...,...,...,...,...
180,583672103,837.946.120-78,Sophie Silveira,+55 21 8211 5733,45.237.196/0001-70
181,438726509,276.901.584-20,Sra. Pietra da Mota,(061) 8931-2556,45.237.196/0001-70
182,436218057,689.125.347-00,Lorena Vieira,+55 (011) 0850-3269,45.237.196/0001-70
183,360748211,306.915.724-61,Pietro da Paz,(061) 4647 5410,45.237.196/0001-70


In [103]:
from datetime import datetime

invoices = pd.DataFrame(columns={"devedorCpf": float, "valor": float,
                                 "dataVencimento": str, "dataPagamento": str})

today = datetime.date(datetime.now())
for index, row in debtors.iterrows():
  first_day = faker.date_between(start_date='-2y', end_date='-1M')
  months = (today - first_day).days // 30
  current_day = first_day + pd.DateOffset(days=0)

  current_invoices = { "devedorCpf": [], "dataVencimento": [],
                      "valor": [], "dataPagamento": [] }

  current_condominium = condominiums[condominiums["cnpj"] == row["cnpjCondominio"]]
  condominium_value = current_condominium["valorMensalidade"].values[0]
  for i in range(months):
    was_paid = faker.pybool()
    valid_payment_date = current_day - pd.DateOffset(days = 30)
    max_valid_payment_date = current_day + pd.DateOffset(days = 60)
    payment_date = faker.date_between(start_date=valid_payment_date,
                                      end_date=max_valid_payment_date)

    current_invoices["devedorCpf"].append(row["cpf"])
    current_invoices["valor"].append(condominium_value)
    current_invoices["dataVencimento"].append(current_day)
    current_invoices["dataPagamento"].append(payment_date if was_paid else None)

    current_day = current_day + pd.DateOffset(months=1)
  
  invoices = pd.concat([invoices, pd.DataFrame(current_invoices)], ignore_index=True)
invoices

C:\Users\danil\AppData\Local\Temp\ipykernel_10928\1893719386.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  invoices = pd.concat([invoices, pd.DataFrame(current_invoices)], ignore_index=True)


,devedorCpf,valor,dataVencimento,dataPagamento
0,768.320.195-21,250,2022-06-04,None
1,768.320.195-21,250,2022-07-04,2022-07-23
2,768.320.195-21,250,2022-08-04,None
3,768.320.195-21,250,2022-09-04,2022-10-31
4,768.320.195-21,250,2022-10-04,2022-11-16
...,...,...,...,...
2364,219.453.708-23,299,2023-04-07,None
2365,219.453.708-23,299,2023-05-07,None
2366,219.453.708-23,299,2023-06-07,2023-05-19
2367,219.453.708-23,299,2023-07-07,None


In [109]:
condominiums.to_csv("./condominiums.csv", index=False)
administrators.to_csv("./administrators.csv", index=False)
debtors.to_csv("./debtors.csv", index=False)
invoices.to_csv("./invoices.csv", index=False)